In [7]:
import geopandas as gpd

census_merge = gpd.read_file("census_merge.gpkg")


In [8]:
filtered = census_merge[
    (census_merge['high_risk']) &
    (census_merge['bufferzone'] == 'Outside') &
    (census_merge['total_pop'] > 0) &
    (census_merge['PCTpov'] > .2) &
    (census_merge['PCTblack'] > .4)
]


In [9]:
import geopandas as gpd

# Ensure it's a GeoDataFrame (with geometry and CRS)
census_merge = gpd.GeoDataFrame(filtered, geometry='geometry', crs=census_merge.crs)

# Export to GeoJSON
census_merge.to_file("filtered_high_risk.geojson", driver="GeoJSON")


In [10]:
import geopandas as gpd
import folium
from pyproj import Transformer

# Load the GeoJSON file
gdf = gpd.read_file("filtered_high_risk.geojson")

# Ensure geometries are valid
gdf = gdf[gdf.geometry.notnull()]

# Reproject to Web Mercator for accurate centroid calculations
projected = gdf.to_crs(epsg=3857)
centroid = projected.geometry.centroid.unary_union.centroid

# Convert centroid back to lat/lon
transformer = Transformer.from_crs(3857, 4269, always_xy=True)
lon, lat = transformer.transform(centroid.x, centroid.y)

# Create folium map with Esri Topographic basemap
m = folium.Map(
    location=[lat, lon],
    zoom_start=12,
    tiles="https://server.arcgisonline.com/ArcGIS/rest/services/World_Topo_Map/MapServer/tile/{z}/{y}/{x}",
    attr="Tiles © Esri — Source: USGS, NOAA, NPS"
)

# Add GeoJSON with tooltip and popup
folium.GeoJson(
    gdf,
    name="Filtered High Risk Areas",
    style_function=lambda feature: {
        'fillColor': 'red',
        'color': 'black',
        'weight': 1,
        'fillOpacity': 0.6,
    },
    tooltip=folium.GeoJsonTooltip(
        fields=["geocode", "como.rate", "total_pop", "med_HH_Income", "PCTpov", "PCTvacant", "PCTnovehicle", "PCTinternetaccess", "PCTPublic35_64", "PCTblack", "PCTwhite", "PCThispanic", "heat_coverage", "veg_cat","pred_como.rate" ],
        aliases=["Geocode", "Real comorbidity Rate", "Population", "Median Household Income", "% Poverty", "% Vacant", "% no vehicle", "% internet access", "% with Public Insurance (35-64)", "% black", "% white", "% hispanic", "Land Coverage Index", "Green Index (gli)", "Predicted Comorbidity Rate" ],
        localize=True
    ),
    popup=folium.GeoJsonPopup(
        fields=["geocode", "total_pop", "med_HH_Income", "PCTpov", "PCTvacant", "PCTnovehicle", "PCTinternetaccess", "PCTPublic35_64", "PCTblack", "PCTwhite", "PCThispanic" ],
        aliases=["Geocode", "Population", "Median Household Income", "% Poverty", "% Vacant", "% no vehicle", "% internet access", "% with Public Insurance (35-64)", "% black", "% white", "% hispanic" ],
        localize=True,
        max_width=300
    )
).add_to(m)

# Add layer control and save
folium.LayerControl().add_to(m)
m.save("high_risk_topo_map.html")
m